In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
from google.colab import auth
from googleapiclient.discovery import build

# Authenticate to Google Drive
auth.authenticate_user()

service = build('drive', 'v3')

FOLDER_ID = "1J8oI8RJEgmbcRdyHzoMzyz519FaJkuSA"
PDF_BASE_URL = "https://drive.google.com/file/d/{id}/view?usp=sharing"

# --- Fetch all PDFs using pagination ---
all_files = []
page_token = None

while True:
    response = service.files().list(
        q=f"'{FOLDER_ID}' in parents and mimeType='application/pdf' and trashed=false",
        fields="nextPageToken, files(id, name)",
        pageToken=page_token
    ).execute()

    all_files.extend(response.get('files', []))
    page_token = response.get('nextPageToken', None)
    if not page_token:
        break

pdf_links = {f["name"]: PDF_BASE_URL.format(id=f["id"]) for f in all_files}

print(f"✅ Found {len(pdf_links)} PDFs in Drive folder (after pagination).")
print("🔗 Example mappings:")
list(pdf_links.items())[:5]


✅ Found 856 PDFs in Drive folder (after pagination).
🔗 Example mappings:


[('the_code_of_criminal_procedure,_1973.pdf',
  'https://drive.google.com/file/d/1N7mmxnKB6qREkm3XBqCVAECFmCpzbXAt/view?usp=sharing'),
 ('the_code_of_civil_procedure,_1908.pdf',
  'https://drive.google.com/file/d/1CtvoN7pM7Of7Rg-8zru01TXKcUjuQCng/view?usp=sharing'),
 ('2023_11_297_312_EN.pdf',
  'https://drive.google.com/file/d/1hB2H7sT3dncaPhVUr4Uj_LPBNGETLu-x/view?usp=sharing'),
 ('2023_7_887_898_EN.pdf',
  'https://drive.google.com/file/d/1uH9NHeNaaKxbvNi0bNUNWHEWGoVS_l4f/view?usp=sharing'),
 ('2023_16_484_524_EN.pdf',
  'https://drive.google.com/file/d/1xJizftlTiEJe68KL5pdYmLSUCRIB44Ux/view?usp=sharing')]

In [17]:
# Step 2 — Check case and name match between local folder and Drive API list

drive_names = [f['name'] for f in all_files]

print("🔍 Checking case-insensitive match between local and Drive filenames:\n")
for f in [
    "2023_16_1209_1524_EN.pdf",
    "2023_14_871_889_EN.pdf",
    "2023_13_832_846_EN.pdf",
    "2023_5_165_214_EN.pdf",
    "2023_14_1073_1082_EN.pdf",
    "2023_3_552_563_EN.pdf"
]:
    match = any(f.lower() == dn.lower() for dn in drive_names)
    if match:
        print(f"✅ {f} — Found in Drive listing")
    else:
        print(f"❌ {f} — Not returned by Drive API (possible rename/mime mismatch)")


🔍 Checking case-insensitive match between local and Drive filenames:

✅ 2023_16_1209_1524_EN.pdf — Found in Drive listing
✅ 2023_14_871_889_EN.pdf — Found in Drive listing
✅ 2023_13_832_846_EN.pdf — Found in Drive listing
✅ 2023_5_165_214_EN.pdf — Found in Drive listing
✅ 2023_14_1073_1082_EN.pdf — Found in Drive listing
✅ 2023_3_552_563_EN.pdf — Found in Drive listing


In [3]:
!pip uninstall -y langchain langchain-core langchain-community langsmith faiss-cpu
!pip install -U pip setuptools wheel

!pip install \
  "langchain==0.1.9" \
  "langchain-core==0.1.53" \
  "langchain-community==0.0.38" \
  "langchain-huggingface==0.0.3" \
  "sentence-transformers==2.7.0" \
  "faiss-cpu==1.8.0.post1" \
  "tqdm" \
  "pymupdf==1.24.10" \
  "python-dotenv"

print("✅ Clean compatible versions installed successfully. Please RESTART RUNTIME after this cell.")

Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.79
Uninstalling langchain-core-0.3.79:
  Successfully uninstalled langchain-core-0.3.79
Found existing installation: langsmith 0.4.40
Uninstalling langsmith-0.4.40:
  Successfully uninstalled langsmith-0.4.40
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 28.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 169.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 128.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 201.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 176.4 MB/s  0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.1.2
    Uninstalling sentence-transformers-5.1.2:
      Successfully uninstalled sentence-transformers-5.1.2
   ━━━━━━━━━━━━━━━━━━

✅ Clean compatible versions installed successfully. Please RESTART RUNTIME after this cell.


In [20]:
import langchain, faiss, sentence_transformers, pymupdf
import langchain_core, langchain_community, langchain_huggingface

print("✅ Environment check successful")
print("LangChain:", langchain.__version__)
print("FAISS:", faiss.__version__)
print("SentenceTransformers:", sentence_transformers.__version__)
print("PyMuPDF:", pymupdf.version)


✅ Environment check successful
LangChain: 0.1.9
FAISS: 1.8.0
SentenceTransformers: 2.7.0
PyMuPDF: ('1.24.10', '1.24.9', '20240902000001')


In [21]:
import os
import glob
import zipfile
import gc
from tqdm import tqdm
import fitz  # PyMuPDF

# LangChain + FAISS imports
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

print("✅ Libraries imported successfully.")

# --- CONFIG ---
BASE_DRIVE_PATH = "/content/drive/My Drive/Nyay-Sahayak-Project"  # adjust to your project folder
CORPUS_BASE_PATH = f"{BASE_DRIVE_PATH}/corpus"
PDF_PATH = f"{CORPUS_BASE_PATH}/case_law/supreme_court_2023"
DB_SAVE_PATH = f"{BASE_DRIVE_PATH}/database"

os.makedirs(DB_SAVE_PATH, exist_ok=True)
print("✅ Paths configured.")

✅ Libraries imported successfully.
✅ Paths configured.


In [22]:
# ✅ SECTION EXTRACTION CELL

def extract_text_from_pdf(pdf_path):
    """Extract text safely from PDFs using PyMuPDF."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text("text")
        doc.close()
        return text
    except Exception as e:
        print(f"⚠️ Error reading {pdf_path}: {e}")
        return ""


import re

def parse_statute_sections(doc_name, text):
    """
    Extract full statutory sections (number, title + body) from CPC/CrPC PDFs.
    Avoids counting 'Order/Rule' numbering but still catches all true sections.
    """
    # Normalize text
    text = re.sub(r'\s+', ' ', text.replace('\n', ' ')).strip()

    # --- Improved pattern ---
    section_pattern = re.compile(
        r'(?<!ORDER\s)(?<!RULE\s)'                  # don’t match inside Orders/Rules
        r'\b(?P<section_no>\d{1,3}[A-Z]?)\.\s*'     # section number
        r'(?P<section_text>'                        # capture title + body
        r'(?:[A-Z][^.]{3,}?\.)'                     # title ends at first period
        r'(?:.*?))'                                 # body text (lazy)
        r'(?=(?:\b\d{1,3}[A-Z]?\.\s)|CHAPTER|PART|\Z)',  # until next section/chapter/part/end
        flags=re.DOTALL
    )

    sections = []
    for m in section_pattern.finditer(text):
        no = m.group("section_no").strip()
        content = re.sub(r'\s+', ' ', m.group("section_text")).strip()
        sections.append({
            "doc_name": doc_name,
            "section_no": no,
            "title_and_text": content
        })

    print(f"✅ Extracted {len(sections)} full sections from {doc_name}")
    return sections



In [23]:
import os
import re
import gc
import glob
import fitz  # PyMuPDF
import zipfile
from tqdm import tqdm

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

In [24]:
import os

PDF_DIR = "/content/drive/MyDrive/Nyay-Sahayak-Project/corpus/case_law/supreme_court_2023"
documents = []

# --- Load Judgment PDFs ---
judgment_dir = f"{CORPUS_BASE_PATH}/case_law/supreme_court_2023"
documents = []

for file in os.listdir(judgment_dir):
    if file.endswith(".pdf"):
        full_path = os.path.join(judgment_dir, file)
        text = extract_text_from_pdf(full_path)
        if len(text.strip()) > 100:
            documents.append({
                "doc_name": file,
                "text": text,
                "type": "judgment"
            })

print(f"✅ Loaded {len(documents)} judgment documents")


# --- Statutes ---
for law_pdf in [
    "the_code_of_civil_procedure,_1908.pdf",
    "the_code_of_criminal_procedure,_1973.pdf"
]:
    pdf_path = os.path.join(PDF_DIR, law_pdf)
    text = extract_text_from_pdf(pdf_path)
    sections = parse_statute_sections(law_pdf, text)
    for section in sections:
        documents.append({
            "doc_name": law_pdf,
            "section_no": section["section_no"],
            "title_and_text": section["title_and_text"],
            "type": "statute"
        })

print(f"📚 Total documents collected: {len(documents)}")


✅ Loaded 856 judgment documents
✅ Extracted 2584 full sections from the_code_of_civil_procedure,_1908.pdf
✅ Extracted 1213 full sections from the_code_of_criminal_procedure,_1973.pdf
📚 Total documents collected: 4653


In [25]:
from google.colab import auth
from googleapiclient.discovery import build

# 1️⃣ Authenticate to Google Drive
auth.authenticate_user()
service = build('drive', 'v3')

# 2️⃣ Your public Drive folder ID
FOLDER_ID = "1J8oI8RJEgmbcRdyHzoMzyz519FaJkuSA"

# 3️⃣ Get all PDFs inside the folder
query = f"'{FOLDER_ID}' in parents and mimeType='application/pdf' and trashed=false"
results = service.files().list(q=query, fields="files(id, name)").execute()
files = results.get('files', [])

# 4️⃣ Build filename → link mapping
# --- Build robust Drive link mapping (case-insensitive + trimmed) ---
PDF_BASE_URL = "https://drive.google.com/file/d/{id}/view?usp=sharing"

pdf_links = {
    f["name"].strip().lower(): PDF_BASE_URL.format(id=f["id"])
    for f in all_files
}

print(f"✅ Found {len(pdf_links)} PDFs in Drive folder (after normalization).")
example_items = list(pdf_links.items())[:5]
print("🔗 Example mappings:")
for k, v in example_items:
    print(f"  {k} -> {v}")


print(f"✅ Found {len(pdf_links)} PDFs in Drive folder")
list(pdf_links.items())[:5]  # preview first 5

missing_links = [d["doc_name"] for d in documents if d["doc_name"] not in pdf_links]
print("❌ Missing Google Drive links for:", missing_links[:10])



✅ Found 856 PDFs in Drive folder (after normalization).
🔗 Example mappings:
  the_code_of_criminal_procedure,_1973.pdf -> https://drive.google.com/file/d/1N7mmxnKB6qREkm3XBqCVAECFmCpzbXAt/view?usp=sharing
  the_code_of_civil_procedure,_1908.pdf -> https://drive.google.com/file/d/1CtvoN7pM7Of7Rg-8zru01TXKcUjuQCng/view?usp=sharing
  2023_11_297_312_en.pdf -> https://drive.google.com/file/d/1hB2H7sT3dncaPhVUr4Uj_LPBNGETLu-x/view?usp=sharing
  2023_7_887_898_en.pdf -> https://drive.google.com/file/d/1uH9NHeNaaKxbvNi0bNUNWHEWGoVS_l4f/view?usp=sharing
  2023_16_484_524_en.pdf -> https://drive.google.com/file/d/1xJizftlTiEJe68KL5pdYmLSUCRIB44Ux/view?usp=sharing
✅ Found 856 PDFs in Drive folder
❌ Missing Google Drive links for: ['2023_2_312_325_EN.pdf', '2023_16_1535_1546_EN.pdf', '2023_15_1067_1073_EN.pdf', '2023_16_888_916_EN.pdf', '2023_7_322_346_EN.pdf', '2023_11_623_642_EN.pdf', '2023_6_419_424_EN.pdf', '2023_2_326_338_EN.pdf', '2023_16_377_434_EN.pdf', '2023_8_152_182_EN.pdf']


In [26]:
docs_final = []
for d in documents:
    filename = d.get("doc_name", "").strip()
    pdf_url = pdf_links.get(filename.lower(), None)


    if "section_no" in d:  # Statutory law
        text = d.get("title_and_text", "")
        meta = {
            "source": filename,
            "type": "statute",
            "section_no": d.get("section_no", ""),
            "section_heading": f"Section {d.get('section_no','')}",
            "pdf_url": pdf_url
        }
    else:  # Judgment
        text = d.get("text", "")
        meta = {
            "source": filename,
            "type": "judgment",
            "case_title": d.get("case_title", ""),
            "pdf_url": pdf_url
        }

    if text and len(text.strip()) > 50:
        docs_final.append(Document(page_content=text.strip(), metadata=meta))

print(f"✅ Final document count with links: {len(docs_final)}")

linked_count = sum(1 for d in docs_final if d.metadata.get("pdf_url"))
print(f"🔗 {linked_count}/{len(docs_final)} documents have valid Drive links.")


✅ Final document count with links: 3757
🔗 3757/3757 documents have valid Drive links.


In [27]:
for f in [
    "2023_3_552_563_EN.pdf",
    "2023_5_165_214_EN.pdf",
    "2023_14_871_889_EN.pdf"
]:
    print(f, "->", pdf_links.get(f.strip().lower()))


2023_3_552_563_EN.pdf -> https://drive.google.com/file/d/1gydnf0_umK8rB3bfal6PR_0JdttFzCK6/view?usp=sharing
2023_5_165_214_EN.pdf -> https://drive.google.com/file/d/1uWmN4Xpor97OKZQWEwF1JTnm9aTJoNwZ/view?usp=sharing
2023_14_871_889_EN.pdf -> https://drive.google.com/file/d/1vwRzGiYcZqvnM7_M0jJneULg3bHP_vbv/view?usp=sharing


In [9]:
print(docs_final[0])


page_content='A\nB\nC\nD\nE\nF\nG\nH\n312\nSUPREME COURT REPORTS\n[2023] 2 S.C.R.\n   [2023] 2 S.C.R. 312\n312\nNATIONAL INSURANCE COMPANY LTD.\nv.\n THE CHIEF ELECTORAL OFFICER & ORS.\n(Civil Appeal No.4769 of 2022)\nFEBRUARY 08, 2023\n[SANJAY KISHAN KAUL AND ABHAY S. OKA, JJ.]\nInsurance – MOU entered into between Appellant-Insurance\ncompany and Respondent No.1 to provide insurance cover to the\npersons deployed for election related work for Bihar Legislative\nAssembly Elections in 2000 – Respondent No.2’s husband, a\nConstable died due to sun stroke while performing election duty –\nRespondent No.2 sought compensation in 2008 – Respondent No.1\nacknowledged the eligibility for payment, Single Judge assigned\nthe liability to pay the amount on the Respondent No.1 and the DM\n– Appeal filed by Respondent No.1, Division Bench fastened the\nliability on the appellant – Held: Respondent No.2’s claim was\nbeyond reasonable time period – It was negligence of Respondent\nNo.1 in lodging th

In [28]:
# ✅ FAISS CREATION CELL

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def build_and_save_faiss_db(documents, db_name):
    """Creates FAISS DB from LangChain documents and saves to Drive."""
    if not documents:
        print(f"⚠️ No documents to process for {db_name}.")
        return

    local_db_path = f"/content/faiss_db_{db_name}"
    zip_save_path = os.path.join(DB_SAVE_PATH, f"faiss_db_{db_name}.zip")

    if os.path.exists(local_db_path):
        !rm -rf {local_db_path}

    print(f"📦 Splitting {len(documents)} docs into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
    chunks = text_splitter.split_documents(documents)
    print(f"✅ Total chunks: {len(chunks)}")

    print("🔹 Building FAISS index...")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(local_db_path)

    print("📦 Zipping database for Drive storage...")
    with zipfile.ZipFile(zip_save_path, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(local_db_path):
            for f in files:
                file_path = os.path.join(root, f)
                zipf.write(file_path, os.path.relpath(file_path, local_db_path))

    print(f"✅ FAISS DB saved and zipped → {zip_save_path}")

    del vectorstore, chunks, documents
    gc.collect()


In [29]:
# 🧪 Test for both PDFs before embedding
# from PyPDF2 import PdfReader # Removed PyPDF2 import

def extract_text_from_pdf_test(pdf_path): # Renamed function to avoid conflict
    """Extract text safely from PDFs using PyMuPDF."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text("text")
        doc.close()
        return text
    except Exception as e:
        print(f"⚠️ Error reading {pdf_path}: {e}")
        return ""

pdf_crpc = "/content/drive/My Drive/Nyay-Sahayak-Project/corpus/case_law/supreme_court_2023/the_code_of_criminal_procedure,_1973.pdf"
pdf_cpc = "/content/drive/My Drive/Nyay-Sahayak-Project/corpus/case_law/supreme_court_2023/the_code_of_civil_procedure,_1908.pdf"

text_crpc = extract_text_from_pdf_test(pdf_crpc)
docs_crpc = parse_statute_sections(os.path.basename(pdf_crpc), text_crpc)
print(f"CrPC sections: {len(docs_crpc)}")

text_cpc = extract_text_from_pdf_test(pdf_cpc)
docs_cpc = parse_statute_sections(os.path.basename(pdf_cpc), text_cpc)
print(f"CPC sections: {len(docs_cpc)}")


✅ Extracted 1213 full sections from the_code_of_criminal_procedure,_1973.pdf
CrPC sections: 1213
✅ Extracted 2584 full sections from the_code_of_civil_procedure,_1908.pdf
CPC sections: 2584


In [30]:
# ✅ MAIN EXTRACTION LOOP

pdf_files = glob.glob(f"{PDF_PATH}/*.pdf")
print(f"Found {len(pdf_files)} PDF files in {PDF_PATH}")

documents = []

for pdf_path in tqdm(pdf_files, desc="Extracting PDFs"):
    pdf_name = os.path.basename(pdf_path)
    text = extract_text_from_pdf(pdf_path)

    if "code_of_criminal_procedure" in pdf_name.lower() or "code_of_civil_procedure" in pdf_name.lower():
        print(f"📑 Parsing statute: {pdf_name}")
        documents.extend(parse_statute_sections(pdf_name, text))
    elif len(text.strip()) > 200:
        documents.append(Document(page_content=text, metadata={"source": pdf_name}))

print(f"📚 Total documents collected: {len(documents)}")

build_and_save_faiss_db(docs_final, "FULL_FINAL_FINALL_FR")

print("🎉 Embeddings created and FAISS DB ready!")


Found 856 PDF files in /content/drive/My Drive/Nyay-Sahayak-Project/corpus/case_law/supreme_court_2023


Extracting PDFs: 100%|█████████▉| 855/856 [01:10<00:00,  4.30it/s]

📑 Parsing statute: the_code_of_civil_procedure,_1908.pdf
✅ Extracted 2584 full sections from the_code_of_civil_procedure,_1908.pdf


Extracting PDFs: 100%|██████████| 856/856 [01:11<00:00, 11.95it/s]

📑 Parsing statute: the_code_of_criminal_procedure,_1973.pdf
✅ Extracted 1213 full sections from the_code_of_criminal_procedure,_1973.pdf
📚 Total documents collected: 4651
📦 Splitting 3757 docs into chunks...


✅ Total chunks: 36664
🔹 Building FAISS index...
📦 Zipping database for Drive storage...
✅ FAISS DB saved and zipped → /content/drive/My Drive/Nyay-Sahayak-Project/database/faiss_db_FULL_FINAL_FINALL_FR.zip
🎉 Embeddings created and FAISS DB ready!


In [31]:
problematic_files = set([
    "2023_16_1209_1524_EN.pdf",
    "2023_14_871_889_EN.pdf",
    "2023_13_832_846_EN.pdf",
    "2023_5_165_214_EN.pdf",
    "2023_14_1073_1082_EN.pdf",
    "2023_3_552_563_EN.pdf"
])

for f in problematic_files:
    print(f"{f} ->", pdf_links.get(f))


2023_14_1073_1082_EN.pdf -> None
2023_16_1209_1524_EN.pdf -> None
2023_14_871_889_EN.pdf -> None
2023_13_832_846_EN.pdf -> None
2023_5_165_214_EN.pdf -> None
2023_3_552_563_EN.pdf -> None


In [14]:
import os

local_folder = "/content/drive/MyDrive/Nyay-Sahayak-Project/corpus/case_law/supreme_court_2023"
problematic_files = [
    "2023_16_1209_1524_EN.pdf",
    "2023_14_871_889_EN.pdf",
    "2023_13_832_846_EN.pdf",
    "2023_5_165_214_EN.pdf",
    "2023_14_1073_1082_EN.pdf",
    "2023_3_552_563_EN.pdf"
]

print("🔍 Checking if these files actually exist in local Drive mount:\n")
for f in problematic_files:
    exists = os.path.exists(os.path.join(local_folder, f))
    print(f"{f}: {'✅ Found' if exists else '❌ Missing'}")


🔍 Checking if these files actually exist in local Drive mount:

2023_16_1209_1524_EN.pdf: ✅ Found
2023_14_871_889_EN.pdf: ✅ Found
2023_13_832_846_EN.pdf: ✅ Found
2023_5_165_214_EN.pdf: ✅ Found
2023_14_1073_1082_EN.pdf: ✅ Found
2023_3_552_563_EN.pdf: ✅ Found
